# ISLDS PROJECT--Glass Data Prediction<br>
#### Information and Computing Sciences 2020 Fu Yinmingren

### Data Analysis<br>
There are 283,104 raw glass data with a total of 794 different characteristics, including characteristics such as glass composition (monomer or compound), viscosity, density, Young's modulus, Abbe number, refractive index, transition temperature, etc. For such a huge amount of data and features, data pre-processing is very necessary. For this project, I chose the feature density293K as the response variable and all the oxide features as predictor variables<br>

### Data pre-processing<br>
For the raw data, I did the following pre-processing steps:<br>
(1)Delete feature columns except for Density293K<br>
(2)Delete all non-oxide and monomer columns<br>
(3)Delete rows and columns with all zeros<br>
(4)Delete abnormal data (abnormally large value for one row in column Density293K)<br>

After the above processing, the dataset still has 9w+ data and 100+ features left, next, to remove the redundant features among them and simplify the process, delete the columns with less than 500 data


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pickle
from sklearn.feature_selection import chi2,SelectKBest
from sklearn.model_selection import cross_val_score,ShuffleSplit,RandomizedSearchCV,learning_curve,train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn import metrics

In [ ]:
# Read raw data
raw_data=pd.read_csv("D:\课件\统计学习与数据科学导论\数据预处理\Data_Glasses.csv") 

In [ ]:
# Delete unnecessary data
pre_data=raw_data.drop(raw_data.columns[759:794],axis=1) # Delete property columns except for Density293K
pre_data=pre_data.drop(pre_data.columns[702:758],axis=1) # Delete property columns except for Density293K
pre_data.iloc[0,0]='O'
fin_data=pre_data.loc[:,pre_data.iloc[0].str.contains('O')] # Deletion of non-oxide substances
temp_data=pre_data.iloc[:,702]

fin_data=pd.concat([fin_data,pre_data.iloc[:,702]],axis=1) # Get all the data of the oxide Density293K
fin_data=fin_data.dropna(subset=[fin_data.columns[198]]) # Delete empty rows in column Density293K

fin_data=fin_data.iloc[:,1:] # Change the name of the first row as the column name
fin_data.columns=fin_data.iloc[0]
fin_data=fin_data[1:].reset_index(drop=True)

fin_data=fin_data.loc[(fin_data.sum(axis=1) != 0),:] # Delete data with all rows 0
fin_data=fin_data.drop(fin_data.columns[0],axis=1)

In [ ]:
fin_data.to_csv('preprocess_data_v1.csv',index=False)

In [ ]:
data=pd.read_csv("D:\课件\统计学习与数据科学导论\数据预处理\preprocess_data.csv")
x_colums=data.columns[0:-1]
y_colums=data.columns[-1]

In [ ]:
count=0
test_data=data
for i in x_colums[:]:
    num=0
    temp=data[i]
    for j in range(len(temp)):
        if temp[j] != 0:
            num+=1
    if num<500:
        #print(i,":",num)
        count+=1
        test_data=test_data.drop(i,axis=1)
#print(count)

print(test_data.head)
x_colums=test_data.columns[0:-1]
y_colums=test_data.columns[-1]
total_colums=test_data.columns
print(x_colums)
print(y_colums)

In [ ]:
# Output the pre-processed data
test_data.to_csv('preprocess_data_1.csv',index=False)

After further pre-processing as described above, the dataset is left with 98,373 data and 49 features

### Methods Analysis

In [7]:
data=pd.read_csv("H:\Python_Project\ISLDS\preprocess_data_1.csv") # Read pre-processed data
x_colums=data.columns[0:-1]
y_colums=data.columns[-1]
total_colums=data.columns

Plot histograms and scatter plots for each feature versus response variable

In [ ]:
# Draw histogram
def hist(data):
    data.hist(figsize=(30,30))
    plt.show()

hist(data[total_colums])

<style>
  .image-container {
    text-align: center; /* 图片及标题居中对齐 */
  }
  .image-container img {
    width: 1200px; /* 设置图片宽度为300像素 */
    height: auto; /* 根据宽度自动调整高度 */
  }
  .image-container figcaption {
    font-size: 20px; /* 设置标题文字大小为14像素 */
    font-weight: bold; /* 设置标题文字加粗 */
    margin-bottom: 10px; /* 设置标题上方距离为10像素 */
  }
</style>

<figure class="image-container">
  <figcaption>Histogram Plot</figcaption><img src="hist.png" alt="图片描述">
</figure>

In [ ]:
# Plotting scatter plots
def scatter(data):
    for i in total_colums[:70]:
         plt.scatter(data[i],data['Density293K'])
         plt.xlabel(i)
         plt.ylabel('Density293K')
         plt.show()
scatter(data)

<style>
  .image-container {
    text-align: center; /* 图片及标题居中对齐 */
  }
  .image-container img {
    width: 800px; /* 设置图片宽度为300像素 */
    height: auto; /* 根据宽度自动调整高度 */
    padding-right: 10px; /* 设置图片右侧间隔为10像素 */
  }
  .image-container figcaption {
    font-size: 20px; /* 设置标题文字大小为14像素 */
    font-weight: bold; /* 设置标题文字加粗 */
    margin-bottom: 5px; /* 设置标题上方距离为10像素 */
  }

</style>

<figure class="image-container">
  <figcaption>SiO2</figcaption><img src="sio2.png" alt="图片描述">
  <figcaption>GeO2</figcaption><img src="geo2.png" alt="图片描述">
  <figcaption>Ga2O3</figcaption><img src="ga2o3.png" alt="图片描述">
</figure>


Before using the processed dataset for regression prediction, the dataset is first normalized and then randomly divided into training set test set division<br>

#### （1）Dataset division

In [43]:
# Test set training set division
train_set,test_set=train_test_split(data.to_numpy(),test_size=0.2,random_state=25)
# print(train_set.shape)
# print(test_set.shape)
x_train=train_set[:,0:49]
y_train=train_set[:,-1]
x_test=test_set[:,0:49]
y_test=test_set[:,-1]

#### （2）Data normalization

In [44]:
# Normalization of data
scaler_x = StandardScaler()
x_train = scaler_x.fit_transform(x_train)
x_test = scaler_x.transform(x_test)

scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test = scaler_y.transform(y_test.reshape(-1, 1))

#### Regression prediction using nonlinear SVM


In [ ]:
# Prediction using non-linear SVM
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.1, 0.01, 0.001],
}
SVM_Model=SVR(kernel='rbf',C=10,gamma=0.1)
# SVM_random_cv=RandomizedSearchCV(estimator=SVM_Model,param_distributions=param_grid,scoring='r2',cv=3,n_iter=10,n_jobs=-1)
SVM_Model.fit(x_train,y_train)
preds=SVM_Model.predict(x_test)

preds=scaler_y.inverse_transform(preds.reshape(-1,1))
y_test=scaler_y.inverse_transform(y_test.reshape(-1,1))

print("Best Parameters: ", SVM_Model.best_params_)
print("Best Score: ", SVM_Model.best_score_)

print("R2 score:",metrics.r2_score(preds,y_test))
print("MSE score:",metrics.mean_squared_error(preds,y_test))
print("MAE score:",metrics.mean_absolute_error(preds,y_test))
# Plotting scatter plots
plt.scatter(preds,y_test)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Scatter Plot')
plt.show()
# Plotting residuals plots
residuals = [predicted - actual for predicted, actual in zip(preds, y_test)]
plt.scatter(y_test, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Actual Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

# Preservation of models
with open('SVM_model.pkl', 'wb') as f:
    pickle.dump(SVM_Model, f)

#### Prediction using Random Forest Regression

In [ ]:
# Prediction using Random Forest Regression
n_estimators = np.arange(20,40,step=1) 
max_features = ["auto", "sqrt", "log2"]
max_depth = list(np.arange(20, 40, step=1)) + [None]
min_samples_split = np.arange(2, 10, step=2)
min_samples_leaf = [1, 2, 4] 
bootstrap = [True, False]
param_grid = {
    "n_estimators": n_estimators,
    "max_features": max_features,
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
    "min_samples_leaf": min_samples_leaf,
    "bootstrap": bootstrap,
}
rf=RandomForestRegressor(n_estimators=22,min_samples_leaf=1,min_samples_split=6,max_features="sqrt",max_depth=37,bootstrap=False)
# random_cv=RandomizedSearchCV(rf,param_grid,n_iter=100,cv=3,scoring="r2",n_jobs=-1)
rf.fit(x_train,y_train)
preds=rf.predict(x_test)

preds=scaler_y.inverse_transform(preds.reshape(-1,1))
y_test=scaler_y.inverse_transform(y_test.reshape(-1,1))

# print("Best Parameters: ", rf.best_params_)
# print("Best Score: ", rf.best_score_)

print("R2 score:",metrics.r2_score(preds,y_test))
print("MSE score:",metrics.mean_squared_error(preds,y_test))
print("MAE score:",metrics.mean_absolute_error(preds,y_test))

# Plotting scatter plots
plt.scatter(preds,y_test)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Scatter Plot')
plt.show()
# Plotting residuals plots
residuals = [predicted - actual for predicted, actual in zip(preds, y_test)]
plt.scatter(y_test, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Actual Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

# Preservation of models
with open('Random_Forest_model.pkl', 'wb') as f:
    pickle.dump(rf, f)

### Experimental Results

<style>
  table {
    margin-left: auto;
    margin-right: auto;
    width: 30%; /* 设置表格宽度为80% */
    height: 80px; /* 设置表格高度为200像素 */
    text-align: center; /* 将表格中的文本内容居中对齐 */
  }
  th, td {
    font-size: 20px; /* 设置文字大小为16像素 */
    font-weight: bold; /* 设置文字加粗 */
  }
  caption {
    font-size: 24px; /* 设置标题文字大小为20像素 */
    font-weight: bold; /* 设置标题文字加粗 */
    margin-bottom: 10px;
  }
</style>
<table>
  <caption>Nonlinear SVM regression prediction</caption>
  <tr>
    <th>R2 score</th>
    <th>MSE score</th>
    <th>MAE score</th>
  </tr>
  <tr>
    <td>0.8557</td>
    <td>0.2133</td>
    <td>0.22259</td>
  </tr>
</table>
<style>
  .image-with-padding {
    padding-right: 30px; /* 设置图片右侧间隔为10像素 */
  }
  .component {
    margin-top: 20px; /* 设置组件上方距离为20像素 */
  }
</style>
<div align="center" class="component"><img src='svm_scatter.png'class="image-with-padding" alt="Image 1"><img src='svm_residual.png' alt="Image 2"></div>

<!-- Random forest regression<br>
R2 score: 0.8530944968179189<br>
MSE score: 0.1965843017716448<br>
MAE score: 0.2040847203381215<br>
<div align="center"><img src='forest_scatter.png'><br></div>
<div align="center"><img src='forest_residual.png'><br></div> -->

<style>
  table {
    margin-left: auto;
    margin-right: auto;
    width: 30%; /* 设置表格宽度为80% */
    height: 80px; /* 设置表格高度为200像素 */
    text-align: center; /* 将表格中的文本内容居中对齐 */
  }
  th, td {
    font-size: 20px; /* 设置文字大小为16像素 */
    font-weight: bold; /* 设置文字加粗 */
  }
  caption {
    font-size: 24px; /* 设置标题文字大小为20像素 */
    font-weight: bold; /* 设置标题文字加粗 */
    margin-bottom: 10px;
  }
</style>
<table>
  <caption>Random forest regression</caption>
  <tr>
    <th>R2 score</th>
    <th>MSE score</th>
    <th>MAE score</th>
  </tr>
  <tr>
    <td>0.8678</td>
    <td>0.196</td>
    <td>0.204</td>
  </tr>
</table>
<style>
  .image-with-padding {
    padding-right: 30px; /* 设置图片右侧间隔为10像素 */
  }
  .component {
    margin-top: 20px; /* 设置组件上方距离为20像素 */
  }
</style>
<div align="center" class="component"><img src='forest_scatter.png'class="image-with-padding" alt="Image 1"><img src='forest_residual.png' alt="Image 2"></div>

### Test Interface

Enter the ***file path*** of the test set to get the test results<br><br>


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pickle
from sklearn.feature_selection import chi2,SelectKBest
from sklearn.model_selection import cross_val_score,ShuffleSplit,RandomizedSearchCV,learning_curve,train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn import metrics

file_path="dataset_example.csv"

X_features=['SIO2', 'P2O5', 'ZRO2', 'NA2O', 'AL2O3', 'FE2O3', 'CAO', 'MgO', 'K2O',
       'MnO', 'GEO2', 'Li2O', 'Ta2O5', 'ZNO', 'SRO', 'CdO', 'SnO2', 'B2O3',
       'La2O3', 'Ga2O3', 'Y2O3', 'TIO2', 'Nb2O5', 'PBO', 'WO3', 'Sb2O3',
       'Bi2O3', 'BAO', 'Cr2O3', 'BeO', 'CuO', 'Nd2O3', 'CeO2', 'Cs2O', 'AS2O3',
       'SO3', 'Rb2O', 'MoO3', 'FeO', 'Ag2O', 'TEO2', 'V2O5', 'Sm2O3', 'Gd2O3',
       'Er2O3', 'Yb2O3', 'H2O', 'SnO', 'O']
X_features.insert(0,'DENSITY')

val_data=pd.read_csv(file_path)

val_data=val_data.drop(val_data.columns[21:35],axis=1)
val_data.columns=val_data.iloc[0]
val_data=val_data[1:]
V_colums=val_data.columns

num_rows=val_data.shape[0]# get rows num
num_colums=val_data.shape[1]# get colums num

temp=pd.DataFrame(np.zeros((num_rows+1,50-num_colums)))
val_data=pd.concat([val_data,temp],axis=1,sort=False,ignore_index=False)# fill the val_data
val_data=val_data.drop(val_data.index[-1])

S_val_data=pd.DataFrame(np.zeros((num_rows,50)))
S_val_data.index=range(1,len(S_val_data)+1)
for label in V_colums:
    index=X_features.index(label)
    S_val_data.iloc[:,index]=val_data[label].copy()

# print(S_val_data)

val_x_colums=val_data.columns[1:]
val_y_colums=val_data.columns[0]
val_x_data=val_data[val_x_colums]
val_y_data=val_data[val_y_colums]

val_x_data=val_x_data.to_numpy()
val_y_data=val_y_data.to_numpy()

scaler_x_val = StandardScaler()
scaler_y_val = StandardScaler()

val_x_data = scaler_x_val.fit_transform(val_x_data)
val_y_data = scaler_y_val.fit_transform(val_y_data.reshape(-1, 1))

val_x_data=scaler_x_val.transform(val_x_data)
val_y_data=scaler_y_val.transform(val_y_data.reshape(-1,1))

# SVM 
with open('SVM_model.pkl', 'rb') as f:
    SAVE_SVM = pickle.load(f)

val_y_preds=SAVE_SVM.predict(val_x_data)

val_y_preds=scaler_y_val.inverse_transform(val_y_preds.reshape(-1,1))
val_y_data=scaler_y_val.inverse_transform(val_y_data.reshape(-1,1))

print("SVM R2 score:",metrics.r2_score(val_y_preds,val_y_data))
print("SVM MSE score:",metrics.mean_squared_error(val_y_preds,val_y_data))
print("SVM MAE score:",metrics.mean_absolute_error(val_y_preds,val_y_data))

val_data=pd.read_csv(file_path)

val_data=val_data.drop(val_data.columns[21:35],axis=1)
val_data.columns=val_data.iloc[0]
val_data=val_data[1:]
V_colums=val_data.columns

num_rows=val_data.shape[0]# get rows num
num_colums=val_data.shape[1]# get colums num

temp=pd.DataFrame(np.zeros((num_rows+1,50-num_colums)))
val_data=pd.concat([val_data,temp],axis=1,sort=False,ignore_index=False)# fill the val_data
val_data=val_data.drop(val_data.index[-1])

S_val_data=pd.DataFrame(np.zeros((num_rows,50)))
S_val_data.index=range(1,len(S_val_data)+1)
for label in V_colums:
    index=X_features.index(label)
    S_val_data.iloc[:,index]=val_data[label].copy()

# print(S_val_data)

val_x_colums=val_data.columns[1:]
val_y_colums=val_data.columns[0]
val_x_data=val_data[val_x_colums]
val_y_data=val_data[val_y_colums]

val_x_data=val_x_data.to_numpy()
val_y_data=val_y_data.to_numpy()

scaler_x_val = StandardScaler()
scaler_y_val = StandardScaler()

val_x_data = scaler_x_val.fit_transform(val_x_data)
val_y_data = scaler_y_val.fit_transform(val_y_data.reshape(-1, 1))

val_x_data=scaler_x_val.transform(val_x_data)
val_y_data=scaler_y_val.transform(val_y_data.reshape(-1,1))

# Ramdon Forest
with open('Random_Forest_model.pkl', 'rb') as f:
    SAVE_Random_Forest = pickle.load(f)

val_y_preds=SAVE_Random_Forest.predict(val_x_data)

val_y_preds=scaler_y_val.inverse_transform(val_y_preds.reshape(-1,1))
val_x_data=scaler_y_val.inverse_transform(val_x_data.reshape(-1,1))

print("Random_Forest R2 score:",metrics.r2_score(val_y_preds,val_y_data))
print("Random_Forest MSE score:",metrics.mean_squared_error(val_y_preds,val_y_data))
print("Random_Forest MAE score:",metrics.mean_absolute_error(val_y_preds,val_y_data))
